# Modeling with GWLF-E using the Model My Watershed API

This notebook demos modeling a shape with GWLF-E using the Model My Watershed (MMW) API. For this, you'll need an MMW account, which you can sign up for free of cost at https://modelmywatershed.org. Once signed in, go to My Account (from the top right menu) → Account and copy your MMW API Token, and save it's value below:

In [1]:
MMW_API_URL = 'https://modelmywatershed.org/api'
MMW_API_TOKEN = '<YOUR TOKEN HERE>'

First, you'll need a shape. You can use use GeoJSON or, in case you want to use a standard watershed, use a HUC-8, HUC-10, or HUC-12 code. The Philadelphia-Schuylkill River HUC-12 has the code 020402031008, let's use that. Note that we need to preserve the leading zero, and hence this should be represented as a string, not a number.

In [2]:
HUC = '020402031008'

With this, we have enough to start modeling. The model is run in 2 steps, a **Prepare** step, and a **Run** step. Let's import some libraries that we'll use throughout:

In [3]:
import pprint
import requests

### Prepare

In the Prepare step, we give the API a shape, and it queries a number of different data layers, including land cover, stream data, soil cover, elevation, slope, and more, to generate an input dictionary for the model. At this stage, we can optionally specify `layer_overrides`, in case we want to gather data from different layers than the ones used by default. By default, this uses the NLCD 2019 dataset for Land Cover, and the NHD+ dataset for streams.

The input payload will look like:

In [4]:
DATA = {'huc': HUC}
HEADERS = {'Authorization': f'Token {MMW_API_TOKEN}'}

Now we have can make the request like:

In [5]:
res = requests.post(f'{MMW_API_URL}/modeling/gwlf-e/prepare/', json=DATA, headers=HEADERS)
res.json()

{'job': '97e26c93-3e23-4213-b7bf-34e8fd060046',
 'job_uuid': '97e26c93-3e23-4213-b7bf-34e8fd060046',
 'status': 'started',
 'messages': ['The `job` field will be deprecated in an upcoming release. Please switch to using `job_uuid` instead.',
  'The `job` field will be deprecated in an upcoming release. Please switch to using `job_uuid` instead.']}

We see that the response conists of job details. The MMW API is asynchronous, which means that every action starts a new job, with a unique `job_uuid`, which we can query to see the status of the job. Let's query the above job and see if it finished:

In [6]:
PREPARE_JOB_UUID = res.json()['job_uuid']
res = requests.get(f'{MMW_API_URL}/jobs/{PREPARE_JOB_UUID}/', headers=HEADERS)
pprint.pprint(res.json(), depth=3)

{'error': '',
 'finished': '2022-05-10T19:55:59.171718Z',
 'job_uuid': '97e26c93-3e23-4213-b7bf-34e8fd060046',
 'result': {'AEU': 0.0007084943035853231,
            'AWMSGrPct': 0,
            'AWMSNgPct': 0,
            'Acoef': [0.091,
                      0.091,
                      0.091,
                      0.26,
                      0.26,
                      0.26,
                      0.26,
                      0.26,
                      0.26,
                      0.091,
                      0.091,
                      0.091],
            'AgLength': 414.6217430368374,
            'AgSlope3': 4.770384041984181,
            'AgSlope3To8': 2.970239120480717,
            'AnimalDailyN': [0.44,
                             0.31,
                             1.07,
                             0.85,
                             0.48,
                             0.37,
                             0.28,
                             0.59,
                             0.0],
 

### Run

The `status` of the result above shows that the job is `complete`. The output is under the `result` key. This is a JSON representation of a GMS file, which is used to capture the entire input required for the GWLF-E model. With this result, we can run the model. We can reuse `PREPARE_JOB_UUID` from above:

In [7]:
res = requests.post(
    f'{MMW_API_URL}/modeling/gwlf-e/run/',
    json={'job_uuid': PREPARE_JOB_UUID},
    headers=HEADERS)
RUN_JOB_UUID = res.json()['job_uuid']

And then we can check the `result` of that job to see the final values:

In [8]:
res = requests.get(f'{MMW_API_URL}/jobs/{RUN_JOB_UUID}/', headers=HEADERS)
res.json()

{'job_uuid': 'e738ea22-8b9f-4e50-bfc8-e3b2aa191b47',
 'status': 'complete',
 'result': {'meta': {'NYrs': 30,
   'NRur': 10,
   'NUrb': 6,
   'NLU': 16,
   'SedDelivRatio': 0.11972374125279198,
   'WxYrBeg': 1961,
   'WxYrEnd': 1990},
  'AreaTotal': 8210.8,
  'MeanFlow': 64849251.16010487,
  'MeanFlowPerSecond': 2.0563562645898297,
  'monthly': [{'AvPrecipitation': 7.924799999999999,
    'AvEvapoTrans': 0.5501632983146786,
    'AvGroundWater': 3.918457483654395,
    'AvRunoff': 3.7172997063186015,
    'AvStreamFlow': 8.878720710082073,
    'AvPtSrcFlow': 1.2429635201090756,
    'AvTileDrain': 0.0,
    'AvWithdrawal': 0.0},
   {'AvPrecipitation': 7.248736666666668,
    'AvEvapoTrans': 0.8376727524299129,
    'AvGroundWater': 3.418162634169156,
    'AvRunoff': 3.6944916758425634,
    'AvStreamFlow': 8.235331037852175,
    'AvPtSrcFlow': 1.1226767278404555,
    'AvTileDrain': 0.0,
    'AvWithdrawal': 0.0},
   {'AvPrecipitation': 8.751993333333331,
    'AvEvapoTrans': 2.4215409599807347,
  

By breaking this down a bit, we see that the `result` has a few different sections:

In [9]:
result = res.json()['result']

result['meta']

{'NYrs': 30,
 'NRur': 10,
 'NUrb': 6,
 'NLU': 16,
 'SedDelivRatio': 0.11972374125279198,
 'WxYrBeg': 1961,
 'WxYrEnd': 1990}

The first section, `meta`, holds values for the weather data: `WxYrBeg` and `WxYrEnd` specify the beginning and end of the weather data, and `NYrs` is the count of years. `NLU` is the total number of land use types, with `NRur` being the number of rural, and `NUrb` being the number of urban land use types. `SedDelivRatio` is the sediment delivery ratio.

Next, there's some values for the entire area of interest:

In [10]:
{k: result[k] for k in ('AreaTotal', 'MeanFlow', 'MeanFlowPerSecond')}

{'AreaTotal': 8210.8,
 'MeanFlow': 64849251.16010487,
 'MeanFlowPerSecond': 2.0563562645898297}

The `monthly` array contains modeled average evapotranspiration, groundwater levels, runoff, stream flow, point source flow, tile drain, and withdrawal for the 12 months, starting with January:

In [11]:
result['monthly']

[{'AvPrecipitation': 7.924799999999999,
  'AvEvapoTrans': 0.5501632983146786,
  'AvGroundWater': 3.918457483654395,
  'AvRunoff': 3.7172997063186015,
  'AvStreamFlow': 8.878720710082073,
  'AvPtSrcFlow': 1.2429635201090756,
  'AvTileDrain': 0.0,
  'AvWithdrawal': 0.0},
 {'AvPrecipitation': 7.248736666666668,
  'AvEvapoTrans': 0.8376727524299129,
  'AvGroundWater': 3.418162634169156,
  'AvRunoff': 3.6944916758425634,
  'AvStreamFlow': 8.235331037852175,
  'AvPtSrcFlow': 1.1226767278404555,
  'AvTileDrain': 0.0,
  'AvWithdrawal': 0.0},
 {'AvPrecipitation': 8.751993333333331,
  'AvEvapoTrans': 2.4215409599807347,
  'AvGroundWater': 3.9516276188257184,
  'AvRunoff': 3.6500011554150147,
  'AvStreamFlow': 8.844592294349809,
  'AvPtSrcFlow': 1.2429635201090756,
  'AvTileDrain': 0.0,
  'AvWithdrawal': 0.0},
 {'AvPrecipitation': 8.906509999999999,
  'AvEvapoTrans': 4.122753658162977,
  'AvGroundWater': 4.300639718909367,
  'AvRunoff': 1.020012441590038,
  'AvStreamFlow': 6.523520083185607,
  'A

The above is used to generate Hydrology graphs in Model My Watershed.

The next is `SummaryLoads`, which has summarized information for:

* Total Loads
* Loading Rates
* Mean Annual Concentration
* Mean Low-Flow Concentration



In [12]:
result['SummaryLoads']

[{'Source': 'Total Loads',
  'Unit': 'kg',
  'Sediment': 5449443.346369732,
  'TotalN': 310162.19472073123,
  'TotalP': 44710.41120645465},
 {'Source': 'Loading Rates',
  'Unit': 'kg/ha',
  'Sediment': 663.692130653497,
  'TotalN': 37.77490557810825,
  'TotalP': 5.445317290209803},
 {'Source': 'Mean Annual Concentration',
  'Unit': 'mg/l',
  'Sediment': 84.03247915563007,
  'TotalN': 4.782818446969861,
  'TotalP': 0.6894514648452934},
 {'Source': 'Mean Low-Flow Concentration',
  'Unit': 'mg/l',
  'Sediment': 88.37639906385692,
  'TotalN': 5.320213722209787,
  'TotalP': 0.9585043511908502}]

each of which contains values for nitrogen, phosphorous, and sediments.

Finally, we have `Loads`, which contain values per source of pollutant:

In [13]:
result['Loads']

[{'Source': 'Hay/Pasture',
  'Sediment': 906.4157983340981,
  'TotalN': 14.228528225910752,
  'TotalP': 4.54027544890625},
 {'Source': 'Cropland',
  'Sediment': 339.5240783632542,
  'TotalN': 15.649501742182162,
  'TotalP': 1.799100543191854},
 {'Source': 'Wooded Areas',
  'Sediment': 212.79124106637443,
  'TotalN': 35.51335343479423,
  'TotalP': 2.006691470069048},
 {'Source': 'Wetlands',
  'Sediment': 1.0869708305827872,
  'TotalN': 43.73424842347138,
  'TotalP': 2.302505265375395},
 {'Source': 'Open Land',
  'Sediment': 207.82694777512648,
  'TotalN': 38.85723400030074,
  'TotalP': 0.9186667663928846},
 {'Source': 'Barren Areas',
  'Sediment': 1.385778001625454,
  'TotalN': 3.5256623177820465,
  'TotalP': 0.1184462201334566},
 {'Source': 'Low-Density Mixed',
  'Sediment': 14303.270578379526,
  'TotalN': 389.6824839377943,
  'TotalP': 41.319078536675704},
 {'Source': 'Medium-Density Mixed',
  'Sediment': 147382.60487996537,
  'TotalN': 3980.3396349579652,
  'TotalP': 412.238228996906

### Modifications

Let's pay attention to the numbers for `Low-Density Open Space`:

In [14]:
[l for l in result['Loads'] if l['Source'] == 'Low-Density Open Space'][0]

{'Source': 'Low-Density Open Space',
 'Sediment': 12350.084036306916,
 'TotalN': 336.469298943642,
 'TotalP': 35.6767418636447}

Now let's say we want to modify the input scenario, but moving all the `Open Land` and `Barren Areas` to `Low-Density Mixed`. We can run GWLF-E with an additional `modifications` array. Each item in the array is an object mapping keys to values. The value is the new one to use. The key is either a direct member of the Prepare `result` (e.g. `n64`), or an array with an index (e.g. `Area__6` for `Area[6]`).

In the following, we set `Area__6` (Open Land) and `Area__7` (Barren Areas) to `0`, and add those values to `Area__10` (Low-Density Mixed), whose new total becomes `1068.6` ha:

In [15]:
MODIFICATIONS = [{
    'Area__6': 0,
    'Area__7': 0,
    'Area__10': 1068.6,
}]

When we re-run GWLF-E with the above modifications, we get:

In [16]:
res = requests.post(
    f'{MMW_API_URL}/modeling/gwlf-e/run/',
    json={'job_uuid': PREPARE_JOB_UUID, 'modifications': MODIFICATIONS},
    headers=HEADERS)
RUN_JOB_UUID = res.json()['job_uuid']

In [17]:
res = requests.get(f'{MMW_API_URL}/jobs/{RUN_JOB_UUID}/', headers=HEADERS)
result = res.json()['result']

[l for l in result['Loads'] if l['Source'] == 'Low-Density Open Space'][0]

{'Source': 'Low-Density Open Space',
 'Sediment': 12410.310316501897,
 'TotalN': 339.52751875830893,
 'TotalP': 35.98391208363879}

In the above we see higher Sediment and TotalN numbers, given the increased area of Low-Density Open Space.